In [21]:
from nustar_pysolar import planning, io
import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.time import TimeDelta
import warnings
warnings.filterwarnings('ignore')

# Download the list of occultation periods from the MOC at Berkeley.

## Note that the occultation periods typically only are stored at Berkeley for the *future* and not for the past. So this is only really useful for observation planning.

In [22]:
fname = io.download_occultation_times(outdir='../data/')
print(fname)

../data/NUSTAR.2021_010.SHADOW_ANALYSIS.txt


# Download the NuSTAR TLE archive.

This contains every two-line element (TLE) that we've received for the whole mission. We'll expand on how to use this later.

The `times`, `line1`, and `line2` elements are now the TLE elements for each epoch.

In [23]:
tlefile = io.download_tle(outdir='../data')
print(tlefile)
times, line1, line2 = io.read_tle_file(tlefile)

../data/NuSTAR.tle


# Here is where we define the observing window that we want to use.

Note that tstart and tend must be in the future otherwise you won't find any occultation times and sunlight_periods will return an error.

In [24]:
tstart = '2021-01-14T09:00:00'
tend = '2021-01-14T17:00:00'
orbits = planning.sunlight_periods(fname, tstart, tend)

In [25]:
orbits

[[<Time object: scale='utc' format='isot' value=2021-01-14T09:33:20.000>,
  <Time object: scale='utc' format='isot' value=2021-01-14T10:36:10.000>],
 [<Time object: scale='utc' format='isot' value=2021-01-14T11:10:00.000>,
  <Time object: scale='utc' format='isot' value=2021-01-14T12:12:50.000>],
 [<Time object: scale='utc' format='isot' value=2021-01-14T12:46:40.000>,
  <Time object: scale='utc' format='isot' value=2021-01-14T13:49:20.000>],
 [<Time object: scale='utc' format='isot' value=2021-01-14T14:23:20.000>,
  <Time object: scale='utc' format='isot' value=2021-01-14T15:26:00.000>]]

In [26]:
# Get the solar parameter
from sunpy.coordinates import sun

angular_size = sun.angular_radius(t='now')
dx = angular_size.arcsec
print(dx)

975.3429073796026


In [27]:
pa = planning.get_nustar_roll(tstart, 0)
print(tstart)
print("NuSTAR Roll angle for Det0 in NE quadrant: {}".format(pa))

2021-01-14T09:00:00
NuSTAR Roll angle for Det0 in NE quadrant: 355.5386486370383 deg


In [35]:
# Orbit 1 (AR)
offset = [-250, -300.]*u.arcsec

ind = 1
orbit = orbits[0]
midTime = (0.5*(orbit[1] - orbit[0]) + orbit[0])
sky_pos = planning.get_skyfield_position(midTime, offset, load_path='./data', parallax_correction=True)
print("Orbit: {}".format(ind))
print(f"Solar offset: {offset}")
print("Orbit start: {} Orbit end: {}".format(orbit[0].iso, orbit[1].iso))
print(f'Aim time: {midTime.iso} RA (deg): {sky_pos[0]:8.4f} Dec (deg): {sky_pos[1]:8.4f}')
print("")


# Orbit 2a (AR)
offset = [-250, -300.]*u.arcsec

ind = 2
orbit = orbits[1].copy()
print(orbit)
oldend = orbit[1]
newend = orbit[1] - TimeDelta(15*60., format = 'sec') # 15 minutes early
orbit[1] = newend

midTime = (0.5*(orbit[1] - orbit[0]) + orbit[0])
sky_pos = planning.get_skyfield_position(midTime, offset, load_path='./data', parallax_correction=True)
print(f"Orbit: {ind}a")
print(f"Solar offset: {offset}")
print("Orbit start: {} Orbit end: {}".format(orbit[0].iso, orbit[1].iso))
print(f'Aim time: {midTime.iso} RA (deg): {sky_pos[0]:8.4f} Dec (deg): {sky_pos[1]:8.4f}')
print("")


# Orbit 2b (small thing)
offset = [200, 300.]*u.arcsec
orbit[0] = newend
orbit[1] = oldend
midTime = (0.5*(orbit[1] - orbit[0]) + orbit[0])
sky_pos = planning.get_skyfield_position(midTime, offset, load_path='./data', parallax_correction=True)
print(f"Orbit: {ind}b")
print(f"Solar offset: {offset}")
print("Orbit start: {} Orbit end: {}".format(orbit[0].iso, orbit[1].iso))
print(f'Aim time: {midTime.iso} RA (deg): {sky_pos[0]:8.4f} Dec (deg): {sky_pos[1]:8.4f}')
print("")






# Orbit 3, 4 (PSP Footpoints)
offset = [-500., -550.]*u.arcsec
for oi in [2, 3]:
    ind = oi+1

    orbit = orbits[oi]
    midTime = (0.5*(orbit[1] - orbit[0]) + orbit[0])
    sky_pos = planning.get_skyfield_position(midTime, offset, load_path='./data', parallax_correction=True)
    print("Orbit: {}".format(ind))
    print(f"Solar offset: {offset}")
    print("Orbit start: {} Orbit end: {}".format(orbit[0].iso, orbit[1].iso))
    print(f'Aim time: {midTime.iso} RA (deg): {sky_pos[0]:8.4f} Dec (deg): {sky_pos[1]:8.4f}')
    print("")




Orbit: 1
Solar offset: [-250. -300.] arcsec
Orbit start: 2021-01-14 09:33:20.000 Orbit end: 2021-01-14 10:36:10.000
Aim time: 2021-01-14 10:04:45.000 RA (deg): 296.1508 deg Dec (deg): -21.3547 deg

[<Time object: scale='utc' format='isot' value=2021-01-14T11:10:00.000>, <Time object: scale='utc' format='isot' value=2021-01-14T12:12:50.000>]
Orbit: 2a
Solar offset: [-250. -300.] arcsec
Orbit start: 2021-01-14 11:10:00.000 Orbit end: 2021-01-14 11:57:50.000
Aim time: 2021-01-14 11:33:55.000 RA (deg): 296.2189 deg Dec (deg): -21.3435 deg

Orbit: 2b
Solar offset: [200. 300.] arcsec
Orbit start: 2021-01-14 11:57:50.000 Orbit end: 2021-01-14 12:12:50.000
Aim time: 2021-01-14 12:05:20.000 RA (deg): 296.0904 deg Dec (deg): -21.1820 deg

Orbit: 3
Solar offset: [-500. -550.] arcsec
Orbit start: 2021-01-14 12:46:40.000 Orbit end: 2021-01-14 13:49:20.000
Aim time: 2021-01-14 13:18:00.000 RA (deg): 296.3756 deg Dec (deg): -21.3943 deg

Orbit: 4
Solar offset: [-500. -550.] arcsec
Orbit start: 2021-0

In [47]:
test1 = SkyCoord(289.3792274160115,  -22.304595055979675, unit = 'deg')
orb1 = SkyCoord(289.3855, -22.3051, unit = 'deg')
orb1.separation(test1)

<Angle 0.00582518 deg>

In [10]:
orbit

[<Time object: scale='utc' format='isot' value=2021-01-14T11:10:00.000>,
 <Time object: scale='utc' format='isot' value=2021-01-14T12:12:50.000>]

In [38]:
import sunpy

In [39]:
sunpy.__version__

'2.0.1'

In [49]:
test1 = SkyCoord(289.898451566591, -22.158432904027155 , unit = 'deg')
orb1 = SkyCoord(289.9047,  -22.1589, unit = 'deg')
orb1.separation(test1)

<Angle 0.00580576 deg>